<a href="https://colab.research.google.com/github/zukol/dw_matrix/blob/master/scraper_lento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import requests
from slugify import slugify
from bs4 import BeautifulSoup
import json
import pandas as pd


In [79]:
class PageScrapper:

    def __init__(self, url):

        self.url = url
        self.last_page = self.get_last_page_number() #pobiera ile jest stron z ogloszeniami

        print(f"Total number of pages: {self.last_page}")

    def get_last_page_number(self):
        page = self.read_page_content(self.url)
        last_page_element = page.find('span', {"class": "hash number alike button"}).get_text() #znajdujemy element span z klasa nazywajaca sie hash number alike button i pobieramy z tego tekst
        return int(last_page_element)
    

    def find_advertisements(self):

        for page_number in range(self.last_page + 1): #iterujemy od 1 od ostatniej strony

            print(f"Page {page_number+1}/{self.last_page} processing...") # pokazuje na ktorej aktualnie stronie jetesmy

            page = self.read_page_content(self.url + f'&page={page_number}') #dopisuje do adresu url kawalek adresu z nr strony
            advertisements = page.findAll("a", {"class": "title-list-item"}) #wyszukujemy wszystkie nazwy ogloszen

            for advertisement_index in range(len(advertisements)):
                data = self.parse_advertisement(advertisements[advertisement_index]['href']) #wchodzimy w petli pokolei w ogloszenia i czytamy tresc za pomaca parse_advertisement
                print(data)
                time.sleep(1) #przerwa 1s zeby nie zostac zablokowanym

    def parse_advertisement(self, url):

        data = {}

        page = self.read_page_content(url)
        container = page.find('div', {'class': 'container-fluid otherpage-container'}) #pobieramy cala strone ogloszenia, ale bez reklam, menu, itp

        data['nazwa'] = container.find('h2').get_text().strip() #pobieramy nazwe ogloszenia za pomoca naglowka h2, metoda strip usuwa biale znaki
        data['tresc'] = container.find('div', {'class': 'desc text-15'}).get_text().strip() #pobieramy tresc ogloszenia
        data['cena'] = float(container.find('div', {'class': 'price'})
                             .get_text().replace('zł', '').replace(' ', '').replace(',', '.')) #usuwamy zl spacje zastepujemy , na .

        parameters = container.find('div', {'class': 'details text-15'}) #wskazujemy gdzie znajduja sie parametry

        for li in parameters:
            label = parameters.find('span', {'class': 'label'}) #dla kazdego elementu li odczytujemy label i value
            value = parameters.find('span', {'class': 'row-old'})

            if value is not None:
                data[slugify(label.get_text())] = value.get_text() # biblioteka ktora tworzy adresy url z tytulow, usuwa polskie znaki itp

        return data

    def read_page_content(self, url):
        page = requests.get(url)
        return BeautifulSoup(page.content, "html.parser")

    def save_data_json(self):
      with open('data_lento.json', 'w') as f:
        json.dump(self.find_advertisements, f)


URL = 'https://gdansk.lento.pl/nieruchomosci/mieszkania.html?price_from=100000'
scraper = PageScrapper(URL)
scraper.find_advertisements()
#scraper.save_data_json()

Total number of pages: 14
Page 1/14 processing...
{'nazwa': 'Mieszkanie Gdańsk Brzeźno, Łozy', 'tresc': 'Opis oferty:Na sprzedaż oferuję 2-pokojowe, komfortowe mieszkanie w nadmorskiej dzielnicy Gdańska - Brzeźnie.\n\nNieruchomość o powierzchni ponad 46 m2 mieści się  na drugim piętrze w pięciokondygnacyjnym bloku położonym w cichej i spokojnej okolicy.\n\nOdległość kilkuminutowego spaceru dzieli nas od licznych atrakcji nadmorskiego kurortu: licznych kawiarenek, restauracji, barów, parku Reagana oraz zadbanej plaży, wzdłuż której ciągnie się promenada z trasami rowerowymi oraz molo w Brzeźnie.\n\nMieszkanie po kapitalnym remoncie,nowocześnie zaaranżowane, pachnące jeszcze świeżością..\n\nW jego skład wchodzą:\n-salon (20m2) z wyjściem na słoneczny balkon od strony płd-zach.\n-sypialnia (11 m2)\n-garderoba (2m2)\n-łazienka z prysznicem (3 m2)\n-kuchnia (6 m2) , \n-przedpokój (4 m2)\n\nDo lokalu przynależy piwnica, a czynsz jest bardzo niski - całorocznie 350 zł.( miejskie ogrzewanie i 

KeyboardInterrupt: ignored

In [67]:
ls

data_lento.json  drive/  sample_data/


In [75]:
df = pd.read_json('data_lento.json', lines=True)
df

""
